In [1]:
import org.apache.spark.sql.SparkSession
import spark.implicits._

import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.linalg.{Vector, Vectors}

Intitializing Scala interpreter ...

Spark Web UI available at http://0637b8c1019d:4040
SparkContext available as 'sc' (version = 2.4.2, master = local[*], app id = local-1559694022834)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import spark.implicits._
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.linalg.{Vector, Vectors}


In [2]:
val df = spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("../Loan_2017_20k.csv")

print((df.count(), df.columns.length))

2019-06-05 00:20:41,355 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
(20070,86)

df: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 84 more fields]


In [11]:
// https://towardsdatascience.com/feature-encoding-with-spark-2-3-0-part-1-9ede45562740

//val categorical_features = df.columns.filter(_.contains("status"))

/*
val categorical_features = Array(
    "verification_status", 
    "loan_status", 
    "initial_list_status",
    "home_ownership",
    "acc_now_delinq",
)
*/

val categorical_features = Array(
    "emp_length", "term", "zip_code", "grade", "sub_grade", "disbursement_method", "title", "purpose", 
    "application_type", "addr_state", "home_ownership", "pymnt_plan", "hardship_flag", 
    "debt_settlement_flag", "verification_status", "initial_list_status",
)

val encodedFeatures = categorical_features.flatMap{ name =>
    
    val stringIndexer = new StringIndexer()
      .setInputCol(name)
      .setOutputCol(name + "_index")
      .setHandleInvalid("skip") // options are "keep", "error" or "skip"
    
    val oneHotEncoder = new OneHotEncoderEstimator()
      .setInputCols(Array(name + "_index"))
      .setOutputCols(Array(name + "_vec"))
      .setDropLast(false)
    
    Array(stringIndexer, oneHotEncoder)
}

val pipeline = new Pipeline()
  .setStages(encodedFeatures)

val df_transformed = pipeline
  .fit(df)
  .transform(df)

//df_transformed.columns.foreach(println)

categorical_features: Array[String] = Array(emp_length, term, zip_code, grade, sub_grade, disbursement_method, title, purpose, application_type, addr_state, home_ownership, pymnt_plan, hardship_flag, debt_settlement_flag, verification_status, initial_list_status)
encodedFeatures: Array[org.apache.spark.ml.Estimator[_ >: org.apache.spark.ml.feature.OneHotEncoderModel with org.apache.spark.ml.feature.StringIndexerModel <: org.apache.spark.ml.Model[_ >: org.apache.spark.ml.feature.OneHotEncoderModel with org.apache.spark.ml.feature.StringIndexerModel <: org.apache.spark.ml.Transformer with org.apache.spark.ml.param.shared.HasHandleInvalid with org.apache.spark.ml.util.MLWritable] with org.apache.spark.ml.param.shared.HasHandleInvalid with org.apache.spark.ml.util.MLWritable] with org.apach...


In [4]:
df.columns
  .filter(_.contains("status"))
  .toArray
  .foreach(println)

verification_status
loan_status
initial_list_status


In [5]:
df_transformed.columns
  .filter(_.contains("status"))
  .toArray
  .foreach(println)

verification_status
loan_status
initial_list_status
verification_status_index
verification_status_vec
initial_list_status_index
initial_list_status_vec


In [6]:
df_transformed
  .groupBy("loan_status")
  .count()
  .sort("count")
  .show()

+------------------+-----+
|       loan_status|count|
+------------------+-----+
|           Default|    1|
| Late (16-30 days)|   58|
|   In Grace Period|  136|
|Late (31-120 days)|  367|
|       Charged Off| 1660|
|        Fully Paid| 5446|
|           Current|12402|
+------------------+-----+



In [ ]:
df_transformed
  .groupBy("loan_status_index")
  .count()
  .sort("count")
  .show()

In [ ]:
df_transformed.groupBy("loan_status_vec")
  .count()
  .sort("count")
  .show()

In [12]:
val vecFeatures = df_transformed
  .columns.filter(_.contains("vec")).toArray

val vectorAssembler = new VectorAssembler()
  .setInputCols(vecFeatures)
  .setOutputCol("categorical_features")

val pipelineVectorAssembler = new Pipeline()
  .setStages(Array(vectorAssembler))

val result_df = pipelineVectorAssembler
  .fit(df_transformed)
  .transform(df_transformed)

vecFeatures: Array[String] = Array(emp_length_vec, term_vec, zip_code_vec, grade_vec, sub_grade_vec, disbursement_method_vec, title_vec, purpose_vec, application_type_vec, addr_state_vec, home_ownership_vec, pymnt_plan_vec, hardship_flag_vec, debt_settlement_flag_vec, verification_status_vec, initial_list_status_vec)
vectorAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_0e86be1c7220
pipelineVectorAssembler: org.apache.spark.ml.Pipeline = pipeline_71731a42aaef
result_df: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 117 more fields]


In [18]:
//https://towardsdatascience.com/feature-encoding-made-simple-with-spark-2-3-0-part-2-5bfc869a809a

//val numerical_features = Array("annual_inc", "loan_amnt")

val numerical_features = Array(
    "delinq_2yrs", "inq_last_6mths", "pub_rec", "out_prncp", "out_prncp_inv", "total_rec_late_fee", 
    "recoveries", "collection_recovery_fee", "collections_12_mths_ex_med", "acc_now_delinq", "tot_coll_amt", 
    "chargeoff_within_12_mths", "delinq_amnt", "num_accts_ever_120_pd", "num_tl_120dpd_2m", "num_tl_30dpd", 
    "num_tl_90g_dpd_24m", "num_tl_op_past_12m", "pub_rec_bankruptcies", "tax_liens", "policy_code", 
    "acc_open_past_24mths", "num_il_tl", "mort_acc", "num_actv_bc_tl", "num_bc_sats", "num_actv_rev_tl", 
    "num_rev_tl_bal_gt_0", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mths_since_recent_bc", "num_bc_tl", 
    "num_op_rev_tl", "int_rate", "open_acc", "num_sats", "num_rev_accts", "percent_bc_gt_75", "total_acc", 
    "dti", "revol_util", "bc_util", "pct_tl_nvr_dlq", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", 
    "installment", "total_rec_int", "last_pymnt_amnt", "avg_cur_bal", "loan_amnt", "funded_amnt", 
    "funded_amnt_inv", "total_rec_prncp", "total_pymnt", "total_pymnt_inv", "bc_open_to_buy", 
    "total_il_high_credit_limit", "revol_bal", "total_bc_limit", "annual_inc", "total_bal_ex_mort", 
    "total_rev_hi_lim", "tot_cur_bal", "tot_hi_cred_lim",
)

val vectorAssembler2 = new VectorAssembler()
  .setInputCols(numerical_features)
  .setOutputCol("numerical_features")
  .setHandleInvalid("skip") // options are "keep", "error" or "skip"

val pipelineVectorAssembler2 = new Pipeline()
  .setStages(Array(vectorAssembler, vectorAssembler2))

val result_df = pipelineVectorAssembler2
  .fit(df_transformed)
  .transform(df_transformed)

result_df.select("numerical_features", "categorical_features").show()

+--------------------+--------------------+
|  numerical_features|categorical_features|
+--------------------+--------------------+
|[0.0,0.0,0.0,0.0,...|(983,[0,12,222,84...|
|[0.0,1.0,1.0,3889...|(983,[5,12,469,85...|
|[0.0,4.0,0.0,2252...|(983,[0,12,159,85...|
|[1.0,0.0,1.0,0.0,...|(983,[2,12,650,84...|
|[0.0,0.0,0.0,0.0,...|(983,[0,12,118,85...|
|[0.0,1.0,0.0,0.0,...|(983,[5,13,55,848...|
|[0.0,1.0,0.0,0.0,...|(983,[6,13,109,84...|
|[0.0,0.0,0.0,0.0,...|(983,[11,12,96,85...|
|[0.0,1.0,0.0,1171...|(983,[6,12,56,849...|
|[1.0,0.0,0.0,7127...|(983,[4,13,53,849...|
|[0.0,1.0,0.0,0.0,...|(983,[0,13,553,85...|
|[0.0,0.0,0.0,1282...|(983,[4,12,14,851...|
|[1.0,1.0,0.0,1604...|(983,[6,12,17,849...|
|[3.0,1.0,0.0,1637...|(983,[11,13,33,84...|
|[0.0,1.0,2.0,8318...|(983,[10,13,112,8...|
|[0.0,3.0,0.0,0.0,...|(983,[3,12,185,84...|
|[4.0,0.0,0.0,8955...|(983,[4,12,16,849...|
|[0.0,2.0,0.0,2124...|(983,[11,13,354,8...|
|[0.0,0.0,0.0,2710...|(983,[3,12,69,850...|
|[0.0,0.0,0.0,0.0,...|(983,[4,12

numerical_features: Array[String] = Array(delinq_2yrs, inq_last_6mths, pub_rec, out_prncp, out_prncp_inv, total_rec_late_fee, recoveries, collection_recovery_fee, collections_12_mths_ex_med, acc_now_delinq, tot_coll_amt, chargeoff_within_12_mths, delinq_amnt, num_accts_ever_120_pd, num_tl_120dpd_2m, num_tl_30dpd, num_tl_90g_dpd_24m, num_tl_op_past_12m, pub_rec_bankruptcies, tax_liens, policy_code, acc_open_past_24mths, num_il_tl, mort_acc, num_actv_bc_tl, num_bc_sats, num_actv_rev_tl, num_rev_tl_bal_gt_0, mo_sin_rcnt_rev_tl_op, mo_sin_rcnt_tl, mths_since_recent_bc, num_bc_tl, num_op_rev_tl, int_rate, open_acc, num_sats, num_rev_accts, percent_bc_gt_75, total_acc, dti, revol_util, bc_util, pct_tl_nvr_dlq, mo_sin_old_il_acct, mo_sin_old_rev_tl_op, installment, total_rec_int, last_pymnt_am...


In [ ]:
//convert the sparse vector to a dense vector as a fail safe

val sparseToDense = udf((v : Vector) => v.toDense)
val result_df_dense = result_df.withColumn("numerical_features", sparseToDense($"numerical_features"))

/**
# convert the data to dense vector
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1])]).toDF(['features'])
**/

In [19]:
val scaler = new StandardScaler()
  .setInputCol("numerical_features")
  .setOutputCol("scaledFeatures")
  .setWithStd(true)
  .setWithMean(true)

// Normalize each feature to have unit standard deviation.
val scaledData = scaler
  .fit(result_df)
  .transform(result_df)

scaledData.select("numerical_features", "scaledFeatures", "categorical_features").show()

+--------------------+--------------------+--------------------+
|  numerical_features|      scaledFeatures|categorical_features|
+--------------------+--------------------+--------------------+
|[0.0,0.0,0.0,0.0,...|[-0.3490493827899...|(983,[0,12,222,84...|
|[0.0,1.0,1.0,3889...|[-0.3490493827899...|(983,[5,12,469,85...|
|[0.0,4.0,0.0,2252...|[-0.3490493827899...|(983,[0,12,159,85...|
|[1.0,0.0,1.0,0.0,...|[0.76314167361979...|(983,[2,12,650,84...|
|[0.0,0.0,0.0,0.0,...|[-0.3490493827899...|(983,[0,12,118,85...|
|[0.0,1.0,0.0,0.0,...|[-0.3490493827899...|(983,[5,13,55,848...|
|[0.0,1.0,0.0,0.0,...|[-0.3490493827899...|(983,[6,13,109,84...|
|[0.0,0.0,0.0,0.0,...|[-0.3490493827899...|(983,[11,12,96,85...|
|[0.0,1.0,0.0,1171...|[-0.3490493827899...|(983,[6,12,56,849...|
|[1.0,0.0,0.0,7127...|[0.76314167361979...|(983,[4,13,53,849...|
|[0.0,1.0,0.0,0.0,...|[-0.3490493827899...|(983,[0,13,553,85...|
|[0.0,0.0,0.0,1282...|[-0.3490493827899...|(983,[4,12,14,851...|
|[1.0,1.0,0.0,1604...|[0.

scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_e62f074c2482
scaledData: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 119 more fields]


In [20]:
val vectorAssembler3 = new VectorAssembler()
  .setInputCols(Array("categorical_features", "scaledFeatures"))
  .setOutputCol("features")

val pipelineVectorAssembler3 = new Pipeline()
  .setStages(Array(vectorAssembler3))

val result_df = pipelineVectorAssembler3
  .fit(scaledData)
  .transform(scaledData)

result_df.select("features").show()

+--------------------+
|            features|
+--------------------+
|(1047,[0,12,222,8...|
|(1047,[5,12,469,8...|
|(1047,[0,12,159,8...|
|(1047,[2,12,650,8...|
|(1047,[0,12,118,8...|
|(1047,[5,13,55,84...|
|(1047,[6,13,109,8...|
|(1047,[11,12,96,8...|
|(1047,[6,12,56,84...|
|(1047,[4,13,53,84...|
|(1047,[0,13,553,8...|
|(1047,[4,12,14,85...|
|(1047,[6,12,17,84...|
|(1047,[11,13,33,8...|
|(1047,[10,13,112,...|
|(1047,[3,12,185,8...|
|(1047,[4,12,16,84...|
|(1047,[11,13,354,...|
|(1047,[3,12,69,85...|
|(1047,[4,12,72,84...|
+--------------------+
only showing top 20 rows



vectorAssembler3: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_47adbe45661f
pipelineVectorAssembler3: org.apache.spark.ml.Pipeline = pipeline_31820cb1f2ba
result_df: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 120 more fields]


In [21]:
// create the labels column
val labelIndexer = new StringIndexer()
  .setInputCol("loan_status")
  .setOutputCol("label")

val df3 = labelIndexer
  .fit(result_df)
  .transform(result_df)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_c23e880ab2d8
df3: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 121 more fields]


In [22]:
// https://www.bmc.com/blogs/using-logistic-regression-scala-spark/

val model = new LogisticRegression().fit(df3)

val predictions = model.transform(df3)

model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_b69dc4aee2e4, numClasses = 6, numFeatures = 1047
predictions: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 124 more fields]


In [23]:
predictions
  .select ("features", "label", "prediction")
  .show()

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(1047,[0,12,222,8...|  1.0|       1.0|
|(1047,[5,12,469,8...|  0.0|       0.0|
|(1047,[0,12,159,8...|  0.0|       0.0|
|(1047,[2,12,650,8...|  2.0|       2.0|
|(1047,[0,12,118,8...|  1.0|       1.0|
|(1047,[5,13,55,84...|  2.0|       2.0|
|(1047,[6,13,109,8...|  1.0|       1.0|
|(1047,[11,12,96,8...|  1.0|       1.0|
|(1047,[6,12,56,84...|  0.0|       0.0|
|(1047,[4,13,53,84...|  3.0|       0.0|
|(1047,[0,13,553,8...|  2.0|       2.0|
|(1047,[4,12,14,85...|  0.0|       0.0|
|(1047,[6,12,17,84...|  0.0|       0.0|
|(1047,[11,13,33,8...|  0.0|       0.0|
|(1047,[10,13,112,...|  0.0|       0.0|
|(1047,[3,12,185,8...|  1.0|       1.0|
|(1047,[4,12,16,84...|  0.0|       0.0|
|(1047,[11,13,354,...|  0.0|       0.0|
|(1047,[3,12,69,85...|  0.0|       0.0|
|(1047,[4,12,72,84...|  1.0|       1.0|
+--------------------+-----+----------+
only showing top 20 rows



In [24]:
//https://stackoverflow.com/questions/37566321/spark-random-forest-binary-classifier-metrics


val binaryClassificationEvaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setRawPredictionCol("prediction")

def printlnMetric(metricName: String): Unit = {
  println(metricName + " = " + binaryClassificationEvaluator
                                 .setMetricName(metricName)
                                 .evaluate(predictions))
}

printlnMetric("areaUnderROC")
printlnMetric("areaUnderPR")

areaUnderROC = 0.9573017890033131
areaUnderPR = 0.9065998118673585


binaryClassificationEvaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4b0a5cf5f331
printlnMetric: (metricName: String)Unit
